<a href="https://colab.research.google.com/github/sivahariditty/CNN-Experiments/blob/main/SqueezNet_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [31]:
transform_train = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

transform_test = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

In [43]:
batch_size = 8
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


VGG - 16

In [33]:
from torchvision import models

In [34]:
sqnet = models.squeezenet1_0(pretrained=True)

In [35]:
for param in sqnet.parameters():
  param.requires_grad = False

In [36]:
sqnet.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))

In [37]:
for param in sqnet.parameters():
  if(param.requires_grad):
    print(param.shape)

torch.Size([10, 512, 1, 1])
torch.Size([10])


In [38]:
def evaluation(dataloader,model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [41]:
sqnet = sqnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt=optim.SGD(sqnet.parameters(),lr=0.5)

In [45]:
import copy


In [56]:
%%time

def train(model,traindata,epochs=1):
  min_loss = 10000
  for epoch in range(epochs):

      for i, data in enumerate(traindata, 0):

          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)

          opt.zero_grad()

          outputs = sqnet(inputs)
          loss = loss_fn(outputs, labels)
          loss.backward()
          opt.step()

          if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(sqnet.state_dict())
            print("Min loss : %0.2f"%min_loss)
          
          del inputs,labels,outputs
          torch.cuda.empty_cache()

          if i%100 == 0:
            print('Iteration %d loss : %0.2f'%(i,loss.item()))
      print('Epoch: %d/%d' % (epoch, epochs))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs


In [52]:
transform_train = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

transform_test = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

In [57]:
batch_size = 8
trainsetSTL = torchvision.datasets.STL10(root='./data', split='train', download=True, transform=transform_train)
trainloaderSTL = torch.utils.data.DataLoader(trainsetSTL, batch_size=batch_size, shuffle=True)
testsetSTL = torchvision.datasets.STL10(root='./data', split='test', download=True, transform=transform_test)
testloaderSTL = torch.utils.data.DataLoader(testsetSTL, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train(sqnet,trainloaderSTL,1)

In [65]:
print(sqnet.features[12].expand3x3) #nn.Conv2d(64, 256, kernel_size=(3,3), stride=(1,1), padding = (1,1))
sqnet.features[12].expand3x3 = nn.Conv2d(64, 256, kernel_size=(3,3), stride=(1,1), padding = (1,1))
print(sqnet.features[12].expand3x3)

Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [66]:
for param in sqnet.parameters():
  if(param.requires_grad):
    print(param.shape)

torch.Size([256, 64, 3, 3])
torch.Size([256])
torch.Size([10, 512, 1, 1])
torch.Size([10])


In [68]:
sqnet = sqnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt=optim.SGD(sqnet.parameters(),lr=0.5)

In [69]:
train(sqnet,trainloaderSTL,1)

Min loss : 2.30
Iteration 0 loss : 2.30
Iteration 100 loss : 2.30
Iteration 200 loss : 2.30
Iteration 300 loss : 2.30


KeyboardInterrupt: ignored